In [57]:
import domain_adapt_run.domain_adapt_run.constants as constants
cache_folder_name = 'projection_gradient_check'
seed = 42

In [58]:
import numpy as np
np.random.seed(seed)
import python_utils.python_utils.basic as basic
import python_utils.python_utils.caching as caching
log_folder = '%s/%s' % (constants.cache_folder, cache_folder_name)
caching.init(constants.cache_folder, constants.which_hash, basic.archiver(log_folder))
%load_ext autoreload
%autoreload 2

[Errno 17] File exists: '/Users/fultonw/Documents/projects/runs/domain_adapt_run/cache/projection_gradient_check'
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import matplotlib.pyplot as plt
import matplotlib
#plt.ion()
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display_pretty, display_html
import python_utils.python_utils.basic as basic
import pandas as pd
import pdb
import itertools
import functools
import scipy.stats
import cvxopt
import statsmodels.regression
import tensorflow as tf
import scipy.optimize
import domain_adapt.domain_adapt.projection as proj
import sklearn.linear_model
import numdifftools as nd
import pymanopt
import pymanopt.manifolds
import pymanopt.solvers
import domain_adapt.domain_adapt.kernels as kernels
import copy

In [60]:
# helper functions, constants
eps = 1e-8
def error(X, y, U, theta):
    y_hat = X.dot(U).dot(theta)
    return np.sum((y_hat - y)**2)

In [61]:
# define projection dimension
K = 2

In [62]:
# define kernel
ker = kernels.nothing_kernel()
#ker = kernels.rbf_kernel(1.)

In [63]:
# generate some data
N = 75
D = 3
X = np.random.normal(size=(N,D))
U = np.random.normal(size=(D,K))
X_U = ker.projected_K(X, U)
L = X_U.shape[1]
B = np.random.normal(size=L)
B = B / np.linalg.norm(B)
#B = np.ones(shape=D)
#B = np.arange(L)+2
print 'B:', B
y = X_U.dot(B) + np.random.normal(scale=0.2,size=N)
print y.shape
#B_z = np.random.uniform(size=L)
B_z = np.random.normal(size=L)
B_z = B_z / np.linalg.norm(B)
z_prob = proj.sigmoid(X_U.dot(B_z))
z = -1+2*(np.random.uniform(size=N) < z_prob).astype(float)
r = 1.
print z
#print X_U.dot(B)

B: [ 0.97854832  0.20601745]
(75,)
[-1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.  1.
 -1.  1.  1.  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.  1.  1.  1. -1.
 -1.  1.  1. -1.  1. -1. -1.  1.  1. -1.  1.  1. -1. -1. -1. -1.  1.  1.
  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1. -1.  1.  1. -1.
  1. -1.  1.]


In [64]:
# fix data to loss function
fixed_logreg_loss = functools.partial(proj.logreg_loss,X_U,z)
fixed_logreg_loss_grad = functools.partial(proj.logreg_loss_grad,X_U,z)

In [65]:
# gradient check for logistic loss
B_0 = np.random.normal(size=L)
print 'numerical gradient:', scipy.optimize.approx_fprime(B_0,fixed_logreg_loss,eps)
print 'analytical gradient', fixed_logreg_loss_grad(B_0)

numerical gradient: [-15.72796791  16.77133312]
analytical gradient [-15.72796948  16.77133135]


In [66]:
# run sklearn logistic regression
fitter = sklearn.linear_model.LogisticRegression(C=150,fit_intercept=False)
fit = fitter.fit(X_U,z)
#print dir(fit)
print 'B from sklearn', fit.coef_
sklearn.metrics.log_loss(z,fit.predict_proba(X_U),normalize=False)

B from sklearn [[ 0.56218273 -0.11117321]]


46.203814727638914

In [67]:
# run logistic regression
#method='Nelder-Mead'
#method='CG'
#method='Newton-CG'
method='BFGS'
scipy.optimize.minimize(fixed_logreg_loss, B_0, jac=fixed_logreg_loss_grad, method=method)

   status: 0
  success: True
     njev: 10
     nfev: 10
 hess_inv: array([[ 0.03416512, -0.00983913],
       [-0.00983913,  0.06054928]])
      fun: 46.2038144444355
        x: array([ 0.56231802, -0.11125382])
  message: 'Optimization terminated successfully.'
      jac: array([  3.71943282e-06,   1.66814258e-06])

In [68]:
# run logistic regression, function from file
print 'B from my logreg from file', proj.logreg(X_U,z)

B from my logreg from file [ 0.56218273 -0.11117321]


# gradient check for U_ij
#U_0 = np.random.normal(size=(D,K))
U_0 = U
print U_0.shape
test_i = 2
test_j = 1
U_0_plus = copy.deepcopy(U_0)
U_0_plus[test_i,test_j] += eps
U_ij_to_K_grad = (ker.projected_K(X, U_0_plus) - ker.projected_K(X, U_0)) / eps
print 'numerical gradient:'
print U_ij_to_K_grad
print 'analytical gradient:'
print ker.U_i_j_to_projected_K_ks_l_grad(X, U_0, test_i, test_j)

In [69]:
# fix projected logreg function
fixed_U_to_B_opt = functools.partial(proj.U_to_B_opt, X, z, ker)

In [70]:
# gradient check for U_to_B_opt
U_0 = U
print 'numerical gradient for U to B_opt:'
print nd.Jacobian(lambda U: fixed_U_to_B_opt(U.reshape((D,K))))(U_0.flatten()).T.reshape((D,K,L))[0:5,0:5,0:5]
#print nd.Jacobian(lambda U: fixed_U_to_B_opt(U.reshape((D,K))))(U_0.flatten())#.T.reshape((D,N,N))
print '\nanalytical gradient for U to B_opt:'
print proj.U_to_B_opt_grad(X, z, U_0, ker)[0:5,0:5,0:5]

numerical gradient for U to B_opt:
[[[-0.23036079 -0.00977279]
  [ 0.05252474 -0.13608285]]

 [[ 0.17056718  0.29353943]
  [-0.02913196 -0.14909494]]

 [[-0.18723931  0.48106418]
  [ 0.03313424 -0.01805086]]]

analytical gradient for U to B_opt:
[[[-0.23041261 -0.00976182]
  [ 0.05254549 -0.13614056]]

 [[ 0.17058705  0.29364558]
  [-0.02912897 -0.14915256]]

 [[-0.187313    0.48126828]
  [ 0.03314296 -0.01806021]]]


In [14]:
# fix L functions
theta_0 = np.random.uniform(size=K)
theta_fixed_U_to_L = lambda U: proj.U_theta_to_L(r, X, y, z, U, theta_0)
U_fixed_theta_to_L = lambda theta: proj.U_theta_to_L(r, X, y, z, U_0, theta)

In [37]:
# gradient check for L
L, (U_to_L_grad, theta_to_L_grad) = proj.U_theta_to_L_with_grad(r, X, y, z, U_0, theta_0)
print 'U:', U_0
print 'theta:', theta_0
print 'L:', L
print 'numerical gradient for U to L:'
print nd.Jacobian(lambda U: theta_fixed_U_to_L(U.reshape((D,K))))(U_0.flatten()).T.reshape((D,K))
print '\nanalytical gradient for U to L:'
print U_to_L_grad
#print proj.U_to_L_grad(r, X, y, theta_0, z, U_0)
print 'numerical gradient for theta to L:'
print nd.Jacobian(U_fixed_theta_to_L)(theta_0)
print '\nanalytical gradient for theta to L:'
#print proj.theta_to_L_grad(r, X, y, U_0, z, theta_0)
print theta_to_L_grad

U: [[ 0.65648121]
 [-0.49077213]]
theta: [ 0.76259639]
L: 6584.58587837
numerical gradient for U to L:
[[-1003.47985952]
 [-2511.02016428]]

analytical gradient for U to L:
[[-1003.47985952]
 [-2511.02016428]]
numerical gradient for theta to L:
[[ 752.13185351]]

analytical gradient for theta to L:
[ 752.13185351]


In [27]:
# now, add some noise dimensions to X
noise_D = 3
noise_X = np.concatenate((X,np.random.normal(size=(N,noise_D-D))),axis=1)
noise_X.shape

(500, 3)

In [28]:
# define desired projection size
reduc_K = D

In [34]:
# set initial U, theta for testing sufficient dimension reduction
#noise_U = np.random.normal(size=(noise_D,reduc_K))
noise_U = np.concatenate((np.eye(reduc_K), np.zeros((noise_D-reduc_K,reduc_K))), axis=0)
#noise_theta = np.random.normal(size=reduc_K)
noise_theta = B
print 'initial U:'
print noise_U
print 'initial theta:'
print noise_theta

initial U:
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  0.]]
initial theta:
[2 3]


In [35]:
# the things that don't change
#manifold = pymanopt.manifolds.Stiefel(noise_D, reduc_K)
manifold = pymanopt.manifolds.Grassmann(noise_D, reduc_K)
solver = pymanopt.solvers.SteepestDescent(maxiter=25)
#solver = pymanopt.solvers.NelderMead(maxiter=25)
#solver = pymanopt.solvers.ParticleSwarm(maxiter=25)

In [ ]:
# take some steps
for i in xrange(5000):
    print 'error before theta:', error(noise_X,y,noise_U,noise_theta)
    theta_cost = lambda theta: proj.U_theta_to_L(r, noise_X, y, z, noise_U, theta)
    theta_grad = lambda theta: proj.theta_to_L_grad_wrapper(r, noise_X, y, noise_U, z, theta)
    method='BFGS'
    noise_theta = scipy.optimize.minimize(theta_cost, noise_theta, jac=theta_grad, method=method, options={'maxiter':15})['x']
    if i % 100 == 0: print noise_theta
    pdb.set_trace()
    print 'error before U:', error(noise_X,y,noise_U,noise_theta)
    U_cost = lambda U: proj.U_theta_to_L(r, noise_X, y, z, U, noise_theta)
    U_egrad = lambda U: proj.U_to_L_grad_wrapper(r, noise_X, y, noise_theta, z, U)
    U_problem = pymanopt.Problem(manifold=manifold, cost=U_cost, egrad=U_egrad)
    noise_U = solver.solve(U_problem, x=noise_U)
    if i % 100 == 0: print noise_U
    pdb.set_trace()

error before theta: 20.4765425007
inside calculated L: 20.4765425007
[ 2.13775687  2.91881328]
> <ipython-input-40-abba955c70b5>(10)<module>()
-> print 'error before U:', error(noise_X,y,noise_U,noise_theta)
(Pdb) c
error before U: 20.4765425007
 iter		   cost val	    grad. norm
inside calculated L: 20.4765425007
    1	+2.0476542500681781e+01	6.23106015e+00
inside calculated L: 3544.27564692
inside calculated L: 1300.54354123
inside calculated L: 384.279663346
inside calculated L: 114.563219442
inside calculated L: 44.0598935982
inside calculated L: 26.2943728829
inside calculated L: 21.8839233458
inside calculated L: 20.8041966476
inside calculated L: 20.5463014024
inside calculated L: 20.4878989304
inside calculated L: 20.4763393012
inside calculated L: 20.4763393012
    2	+2.0476339301151334e+01	5.81470960e+00
inside calculated L: 20.4756470051
inside calculated L: 20.4756470051
    3	+2.0475647005129915e+01	4.09057583e+00
inside calculated L: 20.4757042233
inside calculated L: 20.4

In [ ]:
# calculate error under found U, theta
noise_U

In [17]:
a=np.array([[[1,2],[3,4]],[[5,6],[7,8]]])
print a
print 'gg'
print a.flatten().reshape(a.shape)
print a.ravel()
a.reshape((2,-1),order='C').T

def a():
    return 5

def b():
    a=a()
    
print a
import sklearn.metrics.pairwise
sklearn.metrics.pairwise.rbf_kernel

[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
gg
[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
[1 2 3 4 5 6 7 8]
<function a at 0x1152b08c0>


<function sklearn.metrics.pairwise.rbf_kernel>